In [1]:
import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
from collections import Counter
from pandarallel import pandarallel
pandarallel.initialize()

import collections
from collections import OrderedDict
import ast

headers={
    'accept': '*/*',
#     'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
sitemaps=[\
'https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml',
'https://www.dw.com/en/sitemap-detail-from-2013-08-01-to-2014-01-01.xml',
'https://www.dw.com/en/sitemap-detail-from-2014-01-01-to-2014-06-01.xml',
'https://www.dw.com/en/sitemap-detail-from-2014-06-01-to-2014-11-01.xml',
'https://www.dw.com/en/sitemap-detail-from-2014-11-01-to-2015-04-01.xml',
'https://www.dw.com/en/sitemap-detail-from-2015-04-01-to-2015-08-01.xml',
'https://www.dw.com/en/sitemap-detail-from-2015-08-01-to-2015-12-01.xml']

df = pd.DataFrame(sitemaps, columns = ['page'])
df

,page
0,https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml
1,https://www.dw.com/en/sitemap-detail-from-2013-08-01-to-2014-01-01.xml
2,https://www.dw.com/en/sitemap-detail-from-2014-01-01-to-2014-06-01.xml
3,https://www.dw.com/en/sitemap-detail-from-2014-06-01-to-2014-11-01.xml
4,https://www.dw.com/en/sitemap-detail-from-2014-11-01-to-2015-04-01.xml
5,https://www.dw.com/en/sitemap-detail-from-2015-04-01-to-2015-08-01.xml
6,https://www.dw.com/en/sitemap-detail-from-2015-08-01-to-2015-12-01.xml


In [3]:
def get_articles(page):
    try:
        coverpage = requests.get(page,headers = headers).content
        time.sleep(2)
        data = xmltodict.parse(coverpage)
        articles = data['urlset']['url']
#         return [str(i) for i in articles]
        return articles
    except:
        return

In [4]:
pandarallel.initialize(nb_workers= 2, progress_bar=True)
df['articles']= None
df['articles']= df['page'].parallel_apply(get_articles)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df = df.explode('articles').reset_index(drop=True)
df

,page,articles
0,https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml,"{'loc': 'https://www.dw.com/en/the-road-to-a-bicycle-friendly-city/a-16878573', 'lastmod': '2013-06-14T08:26:54.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
1,https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml,"{'loc': 'https://www.dw.com/en/how-violets-get-ants-to-spread-their-seeds/a-16870652', 'lastmod': '2013-06-13T15:00:44.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
2,https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml,"{'loc': 'https://www.dw.com/en/iea-carbon-emissions-from-fuel-usage-hit-new-global-record/a-16870664', 'lastmod': '2013-06-10T12:29:04.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
3,https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml,"{'loc': 'https://www.dw.com/en/wheelchair-bound-teen-singer-steals-vietnams-heart/a-16870697', 'lastmod': '2013-06-11T12:46:32.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
4,https://www.dw.com/en/sitemap-detail-from-2013-03-01-to-2013-08-01.xml,"{'loc': 'https://www.dw.com/en/africalink-on-air-10-july-2013/a-16870787', 'lastmod': '2013-06-10T19:46:52.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
...,...,...
61284,https://www.dw.com/en/sitemap-detail-from-2015-08-01-to-2015-12-01.xml,"{'loc': 'https://www.dw.com/en/tesla-to-raise-capital-for-e-car-energy-projects/a-18648121', 'lastmod': '2015-08-13T15:57:00.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
61285,https://www.dw.com/en/sitemap-detail-from-2015-08-01-to-2015-12-01.xml,"{'loc': 'https://www.dw.com/en/russian-officers-on-trial-in-kyiv-after-donbass-arrest/a-18841691', 'lastmod': '2015-11-10T16:00:20.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
61286,https://www.dw.com/en/sitemap-detail-from-2015-08-01-to-2015-12-01.xml,"{'loc': 'https://www.dw.com/en/fifa-provisionally-suspends-sepp-blatter-michel-platini-for-90-days/a-18767980', 'lastmod': '2015-10-08T10:31:12.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"
61287,https://www.dw.com/en/sitemap-detail-from-2015-08-01-to-2015-12-01.xml,"{'loc': 'https://www.dw.com/en/help-for-troubled-spies-how-to-leave-intelligence-agencies/a-18749551', 'lastmod': '2015-09-29T16:34:11.000Z', 'changefreq': 'weekly', 'priority': '0.5'}"


In [6]:
df = pd.json_normalize(df.articles).iloc[:,:2]
df

,loc,lastmod
0,https://www.dw.com/en/the-road-to-a-bicycle-friendly-city/a-16878573,2013-06-14T08:26:54.000Z
1,https://www.dw.com/en/how-violets-get-ants-to-spread-their-seeds/a-16870652,2013-06-13T15:00:44.000Z
2,https://www.dw.com/en/iea-carbon-emissions-from-fuel-usage-hit-new-global-record/a-16870664,2013-06-10T12:29:04.000Z
3,https://www.dw.com/en/wheelchair-bound-teen-singer-steals-vietnams-heart/a-16870697,2013-06-11T12:46:32.000Z
4,https://www.dw.com/en/africalink-on-air-10-july-2013/a-16870787,2013-06-10T19:46:52.000Z
...,...,...
61284,https://www.dw.com/en/tesla-to-raise-capital-for-e-car-energy-projects/a-18648121,2015-08-13T15:57:00.000Z
61285,https://www.dw.com/en/russian-officers-on-trial-in-kyiv-after-donbass-arrest/a-18841691,2015-11-10T16:00:20.000Z
61286,https://www.dw.com/en/fifa-provisionally-suspends-sepp-blatter-michel-platini-for-90-days/a-18767980,2015-10-08T10:31:12.000Z
61287,https://www.dw.com/en/help-for-troubled-spies-how-to-leave-intelligence-agencies/a-18749551,2015-09-29T16:34:11.000Z


In [7]:
df.lastmod = pd.to_datetime(df.lastmod)
df.columns= ['url','date']
df = df.sort_values('date').reset_index(drop=True)
df

,url,date
0,https://www.dw.com/en/german-food-scandals/a-16600917,2013-03-01 00:00:00+00:00
1,https://www.dw.com/en/living-in-tokyo/a-16640136,2013-03-01 00:00:00+00:00
2,https://www.dw.com/en/white-kitchen/a-16640150,2013-03-01 00:00:00+00:00
3,https://www.dw.com/en/a-woman-in-japan/a-16640139,2013-03-01 00:00:00+00:00
4,https://www.dw.com/en/the-soul-of-japan/a-16640142,2013-03-01 00:00:00+00:00
...,...,...
61284,https://www.dw.com/en/all-the-latest-at-the-la-auto-show/av-18887898,2015-12-01 23:30:00+00:00
61285,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,2015-12-01 23:30:00+00:00
61286,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,2015-12-01 23:30:00+00:00
61287,https://www.dw.com/en/drive-it-the-motor-magazine/av-18889433,2015-12-01 23:30:00+00:00


In [8]:
df.to_csv('dw_11.csv', index=False)

In [38]:
# df= pd.read_csv('dw_1.csv', header=0)

# 2. filtering

In [9]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)


with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

find_relevant = lambda x: dict(Counter(relevant_rules.findall(str(x).lower())))
find_exclude = lambda x: dict(Counter(irelevant_rules.findall(str(x).lower())))

In [10]:
pandarallel.initialize(nb_workers= 40, progress_bar=False)
df['url_in'] = df['url'].parallel_apply(find_relevant)
df['url_out'] = df['url'].parallel_apply(find_exclude)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [11]:
head_in = df.url_in.str.len()
head_out = df.url_out.str.len()

remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1))

remove_index = df[remove_index== True].index
# remove_index.shape
df = df[~df.index.isin(remove_index)].reset_index(drop=True)
df

,url,date,url_in,url_out
0,https://www.dw.com/en/german-food-scandals/a-16600917,2013-03-01 00:00:00+00:00,{},{}
1,https://www.dw.com/en/living-in-tokyo/a-16640136,2013-03-01 00:00:00+00:00,{},{}
2,https://www.dw.com/en/white-kitchen/a-16640150,2013-03-01 00:00:00+00:00,{},{}
3,https://www.dw.com/en/a-woman-in-japan/a-16640139,2013-03-01 00:00:00+00:00,{},{}
4,https://www.dw.com/en/the-soul-of-japan/a-16640142,2013-03-01 00:00:00+00:00,{},{}
...,...,...,...,...
53141,https://www.dw.com/en/cuba-restores-travel-restrictions-on-doctors/a-18887871,2015-12-01 23:28:43+00:00,{'restrict': 1},{'travel': 1}
53142,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,2015-12-01 23:30:00+00:00,{},{}
53143,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,2015-12-01 23:30:00+00:00,{},{}
53144,https://www.dw.com/en/drive-it-the-motor-magazine/av-18889433,2015-12-01 23:30:00+00:00,{},{}


In [12]:
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df = df[['url','date']]
# df.sort_values('date')
df

,url,date
0,https://www.dw.com/en/german-food-scandals/a-16600917,2013-03-01
1,https://www.dw.com/en/living-in-tokyo/a-16640136,2013-03-01
2,https://www.dw.com/en/white-kitchen/a-16640150,2013-03-01
3,https://www.dw.com/en/a-woman-in-japan/a-16640139,2013-03-01
4,https://www.dw.com/en/the-soul-of-japan/a-16640142,2013-03-01
...,...,...
53141,https://www.dw.com/en/cuba-restores-travel-restrictions-on-doctors/a-18887871,2015-12-01
53142,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,2015-12-01
53143,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,2015-12-01
53144,https://www.dw.com/en/drive-it-the-motor-magazine/av-18889433,2015-12-01


In [13]:
df.to_csv('dw_12.csv', index=False)

# 4

In [ ]:
df= pd.read_csv()

In [17]:
df1= df.copy()
df1

,url,date
0,https://www.dw.com/en/german-food-scandals/a-16600917,2013-03-01
1,https://www.dw.com/en/living-in-tokyo/a-16640136,2013-03-01
2,https://www.dw.com/en/white-kitchen/a-16640150,2013-03-01
3,https://www.dw.com/en/a-woman-in-japan/a-16640139,2013-03-01
4,https://www.dw.com/en/the-soul-of-japan/a-16640142,2013-03-01
...,...,...
53141,https://www.dw.com/en/cuba-restores-travel-restrictions-on-doctors/a-18887871,2015-12-01
53142,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,2015-12-01
53143,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,2015-12-01
53144,https://www.dw.com/en/drive-it-the-motor-magazine/av-18889433,2015-12-01


In [18]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}
   
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_story(url):
    try:
        article = Article(url,config= config)
        article.download()
        time.sleep(2)
        article.parse()
        return article.text
    except:
        print('error %s'%url)
        return
    
def extract_text(url):
    try:
        article = Article(url,config= config)
        time.sleep(1)
        article.download()
        article.parse()
        tags = get_tags(article.html)
        result= [article.title, article.publish_date, article.text,\
                 article.source_url,article.canonical_link, tags] 
        return pd.Series(result)
    
    except:
        return pd.Series(['error']*3)
    
def get_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    try:
        find_tag = soup.find_all('ul', class_= 'smallList')[0]
        tags = find_tag.find_all('a', href=True)
        tags = [i.text for i in tags]
        tags = set(tags)-set(['Send us your feedback.','Print this page'])
    except:
        tags = None
    return tags

In [19]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size= 16*5
i=resume=0
while i< df1.shape[0]:
    print(i)
    df1.loc[i: i+ batch_size-1, ['title','publish_date','text','source','link', 'tag']]=\
    df1.loc[i: i+ batch_size-1, 'url'].parallel_apply(extract_text).values
    i += batch_size
    if (i-resume)% (batch_size * 100) ==0:
        df1.to_csv('dw_13.csv', index= False)    
df1.to_csv('dw_13.csv', index= False)  

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0
80
160
240
320
400
480
560
640
720
800
880
960
1040
1120
1200
1280
1360
1440
1520
1600
1680
1760
1840
1920
2000
2080
2160
2240
2320
2400
2480
2560
2640
2720
2800
2880
2960
3040
3120
3200
3280
3360
3440
3520
3600
3680
3760
3840
3920
4000
4080
4160
4240
4320
4400
4480
4560
4640
4720
4800
4880
4960
5040
5120
5200
5280
5360
5440
5520
5600
5680
5760
5840
5920
6000
6080
6160
6240
6320
6400
6480
6560
6640
6720
6800
6880
6960
7040
7120
7200
7280
7360
7440
7520
7600
7680
7760
7840
7920
8000
8080
8160
8240
8320
8400
8480
8560
8640
8720
8800
8880
8960
9040
9120
9200
9280
9360
9440
9520
9600
9680
9760
9840
9920
10000
10080
10160
10240
10320
10400
10480
10560
10640
10720
10800
10880
10960
11040
11120
11200
11280
11360
11440
11520
11600
11680
11760
11840
11920
12000
12080
12160
12240
12320
12400
12480
12560
12640
12720
12800
12880
12960
13040
13120
13200
13

In [20]:
df = pd.read_csv('dw_13.csv', header=0)
df

,url,date,title,publish_date,text,source,link,tag
0,https://www.dw.com/en/german-food-scandals/a-16600917,2013-03-01,German food scandals,NaN,"DNA confirms what's on the label is in the box\n\nDNA tests can determine what kind of animal meat is in your sandwich or frozen dinner. The process is relatively quick and simple, and the technology for it is already present in many laboratories. (15.02.2013)",https://www.dw.com,https://www.dw.com/en/german-food-scandals/a-16600917,"{'Food', 'scandal', 'horsemeat', 'eggs', 'strawberries', 'dioxin', 'beef', 'Germany', 'food'}"
1,https://www.dw.com/en/living-in-tokyo/a-16640136,2013-03-01,error,error,error,NaN,NaN,NaN
2,https://www.dw.com/en/white-kitchen/a-16640150,2013-03-01,error,error,error,NaN,NaN,NaN
3,https://www.dw.com/en/a-woman-in-japan/a-16640139,2013-03-01,error,error,error,NaN,NaN,NaN
4,https://www.dw.com/en/the-soul-of-japan/a-16640142,2013-03-01,error,error,error,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
53141,https://www.dw.com/en/cuba-restores-travel-restrictions-on-doctors/a-18887871,2015-12-01,Cuba restores travel restrictions on doctors,NaN,"The Caribbean island nation's communist authorities said Tuesday that from December 7 doctors will again be required to ask permission to travel abroad for private reasons.\n\n""In the last three years, nearly half a million Cubans have traveled to other countries on personal business,"" the government reported, saying it represented an 81 percent surge over the prior three-year period. ""In this context, migration of Cuban health professionals is a concern for the country.""\n\nHavana blames a US program that encourages Cuba's skilled medical workers to abandon posts abroad and emigrate to th...",https://www.dw.com,https://www.dw.com/en/cuba-restores-travel-restrictions-on-doctors/a-18887871,"{'Ecuador', 'doctors', 'migrants', 'medicine', 'Cuba', 'embargo'}"
53142,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,2015-12-01,The new Seat Ibiza Cupra,NaN,"Drive it!\n\nThe new Seat Ibiza Cupra\n\nThe CUPRA models have been an established part of the Seat brand for a decade. Now the Spanish carmaker has opened another chapter in the story, with the new Seat Ibiza Cupra, featuring XDS electronic differential lock and adjustable damping controls.",https://www.dw.com,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,"{'Embed video', 'Drive it!', 'Seat Ibiza Cupra', 'adjustable damping', 'Drive it!\n- The Motor Magazine', 'differential lock', 'Save MP4 file'}"
53143,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,2015-12-01,Powerful yet quiet: The Bentley 8 Litre,NaN,"Drive it!\n\nPowerful yet quiet: The Bentley 8 Litre\n\nBetween 1930 and 1932, exactly 100 handcrafted Bentley 8 Litres were produced. While previous Bentleys were famous for the roar of their motorsport engines, the 8 Litre is more of a strong, silent type.",https://www.dw.com,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,"{'Embed video', 'Drive it!', 'vintage cars', 'Drive it!\n- The Motor Magazine', 'Save MP4 file', 'Bentley 8 Litre', 'Bentley', 'handcrafted'}"
53144,https://www.dw.com/en/drive-it-the-motor-magazine/av-18889433,2015-12-01,Drive it! - The Motor Magazine,NaN,"Drive it!\n\nDrive it! - The Motor Magazine\n\nIn this edition, Drive it! goes to the Los Angeles Auto Show to get a preview of all the latest innovations in the automotive world. It also takes an in-depth look at emissions tests and finds out how this now controversial process really works.",https://www.dw.com,https://www.dw.com/en/drive-it-the-motor-magazine/av-18889433,"{'Embed video', 'Drive it!', 'Save MP4 file', 'Drive it!\n- The Motor Magazine'}"


In [21]:
df.columns

Index(['url', 'date', 'title', 'publish_date', 'text', 'source', 'link',
       'tag'],
      dtype='object')

In [22]:
df = df[~df.text.duplicated()]
df = df[~df.text.str.startswith('We use cookies to improve our service for you')]
df=df[['url', 'title', 'text','tag']]
df

,url,title,text,tag
0,https://www.dw.com/en/german-food-scandals/a-16600917,German food scandals,"DNA confirms what's on the label is in the box\n\nDNA tests can determine what kind of animal meat is in your sandwich or frozen dinner. The process is relatively quick and simple, and the technology for it is already present in many laboratories. (15.02.2013)","{'Food', 'scandal', 'horsemeat', 'eggs', 'strawberries', 'dioxin', 'beef', 'Germany', 'food'}"
1,https://www.dw.com/en/living-in-tokyo/a-16640136,error,error,NaN
6,https://www.dw.com/en/the-coal-rush-in-mozambique/a-16691475,The coal rush in Mozambique,Send us your feedback via SMS\n\nGot an opinion about the stories making headlines? Send us a text at +49-160-9575 9510. International SMS charges apply. Please make sure to include your name and your country. We will sample your texts in our show.,"{'Mozmabique', 'coal'}"
18,https://www.dw.com/en/sede-vacante-the-papal-transition/a-16593609,Sede vacante - The papal transition,"Sede vacante is literally the period of the ""empty seat,"" when the Holy See becomes vacant after the death or resignation of a pope. It is a well-regulated interim period. Following his resignation announcement, Pope Benedict XVIremained in officeuntil the end of February.\n\nAs expected, the German pope did not make any spectacular decisions during that period. Such a move would have surprised Stephan Haering, a Munich-based expert on canon law.\n\n""I do not believe he will appoint more cardinals, but he will observe his regular papal duties,"" Haering told a Munich church radio station sh...","{'Pope Francis', 'empty seat', 'Benedikt XVI', 'Sede vacante', 'Pope'}"
19,https://www.dw.com/en/archery-on-horseback/av-16638438,Archery on Horseback,Germany Today\n\nArchery on Horseback\n\nHelma Neye from the state of Brandenburg has opened a school for mounted archery. It’s a sport where a rider shoots with a bow and arrow at full gallop. It has ancient roots and was once practiced in much of the world. Now it’s experiencing something of a modern-day revival.,"{'Germany Today\n- Window on Germany', 'Germany Today'}"
...,...,...,...,...
53141,https://www.dw.com/en/cuba-restores-travel-restrictions-on-doctors/a-18887871,Cuba restores travel restrictions on doctors,"The Caribbean island nation's communist authorities said Tuesday that from December 7 doctors will again be required to ask permission to travel abroad for private reasons.\n\n""In the last three years, nearly half a million Cubans have traveled to other countries on personal business,"" the government reported, saying it represented an 81 percent surge over the prior three-year period. ""In this context, migration of Cuban health professionals is a concern for the country.""\n\nHavana blames a US program that encourages Cuba's skilled medical workers to abandon posts abroad and emigrate to th...","{'Ecuador', 'doctors', 'migrants', 'medicine', 'Cuba', 'embargo'}"
53142,https://www.dw.com/en/the-new-seat-ibiza-cupra/av-18887902,The new Seat Ibiza Cupra,"Drive it!\n\nThe new Seat Ibiza Cupra\n\nThe CUPRA models have been an established part of the Seat brand for a decade. Now the Spanish carmaker has opened another chapter in the story, with the new Seat Ibiza Cupra, featuring XDS electronic differential lock and adjustable damping controls.","{'Embed video', 'Drive it!', 'Seat Ibiza Cupra', 'adjustable damping', 'Drive it!\n- The Motor Magazine', 'differential lock', 'Save MP4 file'}"
53143,https://www.dw.com/en/powerful-yet-quiet-the-bentley-8-litre/av-18887905,Powerful yet quiet: The Bentley 8 Litre,"Drive it!\n\nPowerful yet quiet: The Bentley 8 Litre\n\nBetween 1930 and 1932, exactly 100 handcrafted Bentley 8 Litres were produced. While previous Bentleys were famous for the roar of their motorsport engines, the 8 Litre is more of a strong, silent type.","{'Embed video', 'Drive it!', 'vintage cars', 'Drive it!\n- The Motor Magazine', 'Save MP4 file', 'Bentley 8 Litre', 'Bentley', 'handcrafte

In [23]:
df.to_csv('dw_13_15.csv', index=False)

#  Merge

In [25]:
df0 = pd.read_csv('dw_01_13.csv', header=0)
df1 = pd.read_csv('dw_13_15.csv', header=0)
df2 = pd.read_csv('dw_16_21.csv', header=0)


In [35]:
df2= df2.rename(columns={'tags':'tag'})

In [36]:
df_all = pd.concat([df0, df1, df2])
df_all = df_all[df_all.text.str.len()>150]
df_all = df_all[~df_all.duplicated('text')]
df_all

,url,title,text,tag,date,authors
0,https://www.dw.com/en/rising-criticism-of-us-security-alert/a-317537,Rising criticism of US security alert,"U.S. President George Bush has defended his decision to put the country on national security alert over the next week.\n\nResponding to rising criticism that the warning was vague and alarmist, Bush said on Wednesday that he had issued it because the government had information that made it necessary to enhance protection of vulnerable areas and assets.\n\nHis statement came as public concern was stoked by the announcement that inhalation anthrax had killed a hospital worker in New York.\n\nThe White House said Wednesday that investigators still did not know who was responsible for the outb...",set(),2001-11-01,NaN
1,https://www.dw.com/en/stressed-out/a-317575,"Arts, music and lifestyle reporting from Germany","Nearly half of all Germans feel stressed-out by everyday life. According to a survey just published by the pollsters Forsa,\n\n45 percent of the population are under constant stress.\n\nFor 35 percent of those surveyed, stress factor number one is - too much work. For 23 percent, it is the problems in their private and family lives that tax their nerves.\n\nFor half of those feeling stressed-out, a relaxed meal is a good way to unwind.",set(),2001-11-01,NaN
2,https://www.dw.com/en/big-spenders/a-317579,"Arts, music and lifestyle reporting from Germany","The Art Cologne event is an artistic as well as a commercial occasion.\n\nOn view are artworks ranging from classical modernism to contemporary art, gathered from the collections of around 270 galleries from 20 different countries.\n\nArt Cologne is thought of as a kind of test case that will show how the global art scene fares in the present climate of crisis and war.",set(),2001-11-01,NaN
3,https://www.dw.com/en/lucky-for-some/a-317584,Lucky for Some,"Here in Wittenberg and all over Germany, thousands of Protestants have come together to commemorate Martin Luther and to celebrate Reformation Day.\n\nMemorial services recall the beginning of the religious renewal movement of the 16th century.\n\nOn October 31, 1517, Martin Luther is said to have nailed his 95 theses on the door of the Wittenberg Palace Church. His reformist ideas split the Church in two, between Catholics and Protestants.\n\nIn most of the eastern German states, Reformation Day is a public holiday. But in the West, people still went to work today.",set(),2001-11-02,NaN
4,https://www.dw.com/en/washington-in-a-state-of-alarm/a-318007,Washington: In a State of Alarm,"The White House has defended this week's national security alert by saying it is needed to protect areas and assets that might be vulnerable.\n\nBut in the face of threats that can turn a letter into a deadly weapon, just about anything is a vulnerable target these days. With even the House of Representatives recently forced to close for decontamination, the US capital is entering what the President this week referred to as an ""unusual period in American history.""\n\nThe US has fought many wars, but what Americans are now calling the ""homeland"" has not been attacked for nearly 200 years. I...",set(),2001-11-02,NaN
...,...,...,...,...,...,...
56479,http://www.dw.com/en/made-in-china-2025-the-next-stage-of-chinas-economic-rise/a-18490377,'Made in China 2025' - The next stage of China's economic rise?,"Faced with slowing economic growth rates, rising wage costs, and growing competition from other emerging markets, Chinese leaders have been seeking new ways to upgrade the country's manufacturing sector - a key industry that has largely driven China's economic rise over the past three decades.\n\nAnd it seems that those efforts have now yielded major results: Headed by Premier Li Keqiang, the country's cabinet, the State Council, recently unveiled ""Made in China 2025,"" a sweeping national strategy designed to enhance competitiveness in this sector through automation and overall improvement...","{""Peopl

In [38]:
df_all.date = df_all.date.apply(lambda x: str(x).replace('-',''))
df_all.sort_values('date')

,url,title,text,tag,date,authors
0,https://www.dw.com/en/rising-criticism-of-us-security-alert/a-317537,Rising criticism of US security alert,"U.S. President George Bush has defended his decision to put the country on national security alert over the next week.\n\nResponding to rising criticism that the warning was vague and alarmist, Bush said on Wednesday that he had issued it because the government had information that made it necessary to enhance protection of vulnerable areas and assets.\n\nHis statement came as public concern was stoked by the announcement that inhalation anthrax had killed a hospital worker in New York.\n\nThe White House said Wednesday that investigators still did not know who was responsible for the outb...",set(),20011101,NaN
1,https://www.dw.com/en/stressed-out/a-317575,"Arts, music and lifestyle reporting from Germany","Nearly half of all Germans feel stressed-out by everyday life. According to a survey just published by the pollsters Forsa,\n\n45 percent of the population are under constant stress.\n\nFor 35 percent of those surveyed, stress factor number one is - too much work. For 23 percent, it is the problems in their private and family lives that tax their nerves.\n\nFor half of those feeling stressed-out, a relaxed meal is a good way to unwind.",set(),20011101,NaN
2,https://www.dw.com/en/big-spenders/a-317579,"Arts, music and lifestyle reporting from Germany","The Art Cologne event is an artistic as well as a commercial occasion.\n\nOn view are artworks ranging from classical modernism to contemporary art, gathered from the collections of around 270 galleries from 20 different countries.\n\nArt Cologne is thought of as a kind of test case that will show how the global art scene fares in the present climate of crisis and war.",set(),20011101,NaN
10,https://www.dw.com/en/china-and-germany-a-lucrative-relationship/a-318178,China and Germany: A Lucrative Relationship,"On Friday Chancellor Gerhard Schröder wrapped up his whirlwind tour of China by heaping praise on the country's financial standing and promising to strengthen relations between the two countries.\n\nThe Chancellor described his visit to China as positive, and said the bilateral talks had contributed to the development of a relationship based on mutual trust.\n\nA day earlier several of Germany's industrial powerhouses sealed a slew of deals that will ensure Germany's place as China's top European trading partner. The attendance roster was first class: BASF, Siemens, Bayer. They and others ...",set(),20011102,NaN
9,https://www.dw.com/en/threat-to-californian-bridges/a-318175,Threat to Californian Bridges,"California Governor Gray Davis has tightened security around the state's major bridges, saying he has received ""credible"" information that they could be targeted for attack.\n\nHe said that information ""from several different sources"" spoke of a possible attempt to blow up the bridges during the rush hour between November 2 and 7.\n\nHe said that the bridges wouldn't be closed, and it was up to commuters to decide whether to drive over them in the coming days.\n\nThe bridges at risk include the Golden Gate and Bay Bridges in San Francisco, the Vincent Thomas Bridge in Los Angeles and the C...",set(),20011102,NaN
...,...,...,...,...,...,...
31847,https://www.dw.com/en/nsa-and-british-spies-hack-into-dutch-company-producing-sim-cards/a-18270000,NSA and British spies 'hack into Dutch company producing SIM cards',"Operatives from Britain and the NSA hacked into the internal network of Dutch company Gemalto, the world's largest manufacturer of SIM cards for mobile phones, The Intercept reported on Thursday, saying it had the information from documents provided by NSA whistleblower Edward Snowden.\n\nGlenn Greenwald, founder of The Intercept and the man who made Snowden's leaks public tweeted the news:\n\nAccess to private information\n\nGemalto produces two billion SIM cards every year. Its clients include T Mobile, Verizon and AT&T

In [42]:
df_all = df_all.iloc[:,[4,1,2,3,0]]
df_all

,date,title,text,tag,url
0,20011101,Rising criticism of US security alert,"U.S. President George Bush has defended his decision to put the country on national security alert over the next week.\n\nResponding to rising criticism that the warning was vague and alarmist, Bush said on Wednesday that he had issued it because the government had information that made it necessary to enhance protection of vulnerable areas and assets.\n\nHis statement came as public concern was stoked by the announcement that inhalation anthrax had killed a hospital worker in New York.\n\nThe White House said Wednesday that investigators still did not know who was responsible for the outb...",set(),https://www.dw.com/en/rising-criticism-of-us-security-alert/a-317537
1,20011101,"Arts, music and lifestyle reporting from Germany","Nearly half of all Germans feel stressed-out by everyday life. According to a survey just published by the pollsters Forsa,\n\n45 percent of the population are under constant stress.\n\nFor 35 percent of those surveyed, stress factor number one is - too much work. For 23 percent, it is the problems in their private and family lives that tax their nerves.\n\nFor half of those feeling stressed-out, a relaxed meal is a good way to unwind.",set(),https://www.dw.com/en/stressed-out/a-317575
2,20011101,"Arts, music and lifestyle reporting from Germany","The Art Cologne event is an artistic as well as a commercial occasion.\n\nOn view are artworks ranging from classical modernism to contemporary art, gathered from the collections of around 270 galleries from 20 different countries.\n\nArt Cologne is thought of as a kind of test case that will show how the global art scene fares in the present climate of crisis and war.",set(),https://www.dw.com/en/big-spenders/a-317579
3,20011102,Lucky for Some,"Here in Wittenberg and all over Germany, thousands of Protestants have come together to commemorate Martin Luther and to celebrate Reformation Day.\n\nMemorial services recall the beginning of the religious renewal movement of the 16th century.\n\nOn October 31, 1517, Martin Luther is said to have nailed his 95 theses on the door of the Wittenberg Palace Church. His reformist ideas split the Church in two, between Catholics and Protestants.\n\nIn most of the eastern German states, Reformation Day is a public holiday. But in the West, people still went to work today.",set(),https://www.dw.com/en/lucky-for-some/a-317584
4,20011102,Washington: In a State of Alarm,"The White House has defended this week's national security alert by saying it is needed to protect areas and assets that might be vulnerable.\n\nBut in the face of threats that can turn a letter into a deadly weapon, just about anything is a vulnerable target these days. With even the House of Representatives recently forced to close for decontamination, the US capital is entering what the President this week referred to as an ""unusual period in American history.""\n\nThe US has fought many wars, but what Americans are now calling the ""homeland"" has not been attacked for nearly 200 years. I...",set(),https://www.dw.com/en/washington-in-a-state-of-alarm/a-318007
...,...,...,...,...,...
56479,20150621,'Made in China 2025' - The next stage of China's economic rise?,"Faced with slowing economic growth rates, rising wage costs, and growing competition from other emerging markets, Chinese leaders have been seeking new ways to upgrade the country's manufacturing sector - a key industry that has largely driven China's economic rise over the past three decades.\n\nAnd it seems that those efforts have now yielded major results: Headed by Premier Li Keqiang, the country's cabinet, the State Council, recently unveiled ""Made in China 2025,"" a sweeping national strategy designed to enhance competitiveness in this sector through automation and overall improvement...","{""People's Republic of China"", 'China', 'Made in China 2025', 'industry', 'economy', 'Germany', 'Beijing', 'Asia'}",http://www.dw

In [18]:
df_all.to_csv('dw_20')

,url,date,title,authors,text,tags
0,https://www.dw.com/en/atlanta-shootings-hundreds-rally-to-support-asian-americans/a-56941880,20210321,Atlanta shootings: Hundreds rally to support Asian-Americans,"['Deutsche Welle', 'Www.Dw.Com']","Hundreds of people rallied outside the Georgia State Capitol in Atlanta on Saturday to denounce violence against the Asian-American community after eight people, including six women of Asian descent, were killed in shootings at three spas earlier this week.\n\nDemonstrators of all age groups and ethnic backgrounds were seen waving American flags and carrying placards that read ""We are not the virus"" and ""Stop Asian Hate.""\n\nUS Senators Raphael Warnock and Jon Ossoff and Georgia state Representative Bee Nguyen — the first Vietnamese-American to serve in the Georgia House — were among those...","{'Atlanta', 'Hate Crime', 'Asian-Americans', 'White House', 'Racism', 'Atlanta shooting', 'United States'}"
1,https://www.dw.com/en/israel-thousands-protest-against-pm-netanyahu-ahead-of-polls/a-56942273,20210321,Israel: Thousands protest against PM Netanyahu ahead of polls,"['Deutsche Welle', 'Www.Dw.Com']","Thousands of people in Israel held demonstrations outside the residence of Prime Minister Benjamin Netanyahu in Jerusalem on Saturday, calling for his ouster with only three days left for the country's fourth parliamentary election in two years.\n\nPeople waved flags and carried posters about the need for a ""revolution"" and mistrust in the police, while a man with a bullhorn was heard chanting ""Bibi go home,"" using Netanyahu's nickname.\n\nThe anti-Netanyahu protests have been a weekly occurrence for the last nine months.\n\nHowever, they saw a decline in numbers due to the winter and afte...","{'Israel', 'Benjamin Netanyahu', 'Israel elections', 'Fatah', 'Jerusalem'}"
2,https://www.dw.com/en/coronavirus-digest-pakistan-pm-khan-tests-covid-positive/a-56942320,20210321,Coronavirus digest: Pakistan PM Khan tests COVID positive,"['Deutsche Welle', 'Www.Dw.Com']","Pakistani Prime Minister Imran Khan has tested positive for the coronavirus two days after he received his first vaccine dose, officials in Islamabad have said.\n\nFaisal Sultan, Khan's special assistant on health, said the prime minister has quarantined himself at his private home in the country's capital city.\n\nThe government said the vaccine had not been effective because Khan had received it so recently, and it was only the first of two required doses.\n\nThere has been a spike in COVID-19 cases in the capital and in eastern and northern Pakistan.\n\nAuthorities on Saturday reported ...","{'Health', 'Coronavirus', 'health', 'coronavirus'}"
3,https://www.dw.com/en/brazils-jair-bolsonaro-under-pressure-over-coronavirus-policy/a-56941250,20210321,Brazil's Jair Bolsonaro under pressure over coronavirus policy,"['Deutsche Welle', 'Www.Dw.Com']","In the dead of night, a German army transport plane carrying 80 ventilators lifted off from Cologne airport en route to Manaus, Brazil. The devices were desperately needed, as the health-care system in the Amazon metropolis had once again collapsed under a rush of COVID-19 patients.\n\nLast April, not quite a year ago, the world was shocked to see images of scores of dead being buried in mass graves because Manaus' cemeteries were filled beyond capacity. This January, patients with serious COVID-19 related illnesses lacked oxygen supplies.\n\nBrazil has been a global coronavirus hotspot fo...","{'Coronavirus', 'Brazil', 'Lula da Silva', 'coronavirus', 'Jair Bolsonaro', 'COVID-19'}"
4,https://www.dw.com/en/philippines-220-chinese-militia-boats-gathered-in-south-china-sea/a-56943127,20210321,Philippines asks Chinese flotilla to leave disputed reef,"['Deutsche Welle', 'Www.Dw.Com', 'Author', 'Rodion Ebbighausen', 'Wr']","On Sunday, the Philippine defense chief demanded that more than 200 Chinese vessels he said were manned by militias leave a South China Sea reef claimed by both Manila and Beijing.

In [23]:
df_others[df_others.date.apply(str).str.contains('2015')]

,url,date,title,authors,text,tags
50780,http://www.dw.com/en/nigeria-ready-to-negotiate-with-boko-haram-for-chibok-girls-release-says-president/a-18952332,20151231,"Nigeria ready to negotiate with Boko Haram for Chibok girls' release, says president","['Deutsche Welle', 'Www.Dw.Com']","""We are prepared to negotiate without conditions with credible Boko Haram leadership,"" Nigerian President Muhammadu Buhari told reporters in a media chat broadcast on Wednesday.\n\nIn April 2014, 276 girls were kidnapped by the Islamist militant group Boko Haram in the northeastern Nigerian town of Chibok, sparking headlines across the globe.\n\nWhile a handful of the girls were able to escape captivity shortly after being kidnapped, some 200 of them have not been seen since a video of them was released in May 2014.\n\nBuhari noted that negotiations could not begin until Nigerian authoriti...","{'Chibok', 'Muhammadu Buhari', 'Boko Haram', 'Nigeria'}"
50781,http://www.dw.com/en/san-bernardino-gunmans-neighbor-faces-additional-charges/a-18952361,20151231,San Bernardino gunman's neighbor faces additional charges,"['Deutsche Welle', 'Www.Dw.Com']","The 24-year-old was charged on Wednesday with conspiring with the San Bernardino gunman Syed Farook to carry out attacks in 2011 and 2012 even though the pair never went through with their plans.\n\nMarquez was also accused of making false statements when he bought the guns used in this month's deadly shootings.\n\nThe new charges follow a similar indictment on December 17, where he was charged with three counts of conspiring to commit a terrorist act, making an illegal gun purchase and committing immigration fraud in an alleged sham marriage to a Russian bride.\n\nHistoric threats\n\nIn n...","{'Islamist', 'Marquez', 'San Bernardino', 'US', 'terror attack', 'shootings', 'California'}"
50782,http://www.dw.com/en/twin-attacks-on-syrian-kurdish-city-kill-at-least-16/a-18952412,20151231,Twin attacks on Syrian Kurdish city kill at least 16,"['Deutsche Welle', 'Www.Dw.Com']","Kurdish news outlet Rudraw reported on Wednesday that two bombs were detonated near restaurants in a predominantly Christian quarter of Qamishli, which lies near the border with Turkey and close to Iraq.\n\nThe British-based Syrian Observatory for Human Rights also reported a third explosion, but provided no further information. Observatory director Rami Abdel Rahman told the AFP news agency that at least one of the blasts was caused by a suicide bomber.\n\nSeries of 'IS' attacks\n\nAmaq news agency, which supports the so-called ""Islamic State"" (IS), said shortly after the attacks that the...","{'Syria YPG', 'Kurdish', 'Islamic State', 'Qamishli', 'IS'}"
50783,http://www.dw.com/en/air-canada-flight-diverted-after-heavy-turbulence-injures-21/a-18952432,20151231,Air Canada flight diverted after heavy turbulence injures 21,"['Deutsche Welle', 'Www.Dw.Com']","The Boeing 777-300ER, which was carrying 332 passengers and 19 crew, made the emergency landing at 3:23 p.m. local time (2223 UTC) on Wednesday.\n\n""Confirming AC088 Shanghai to Toronto, encountered turbulence en route, diverted Calgary,"" Air Canada announced on Twitter.\n\nAfter landing in Calgary in southern Canada, the airline said 21 passengers were transferred to hospitals - eight had non-life-threatening injuries, another 13 were sent for observation. Three children were also among the injured.\n\n'Unsettling experience'\n\nExecutive Vice President and Chief Operating Officer at Air ...","{'turbulence', 'flight AC088', 'Shanghai', 'Toronto', 'Air Canada'}"
50784,http://www.dw.com/en/french-journalist-ursula-gauthier-prepares-to-be-effectively-expelled-from-china/a-18952536,20151231,French journalist Ursula Gauthier prepares to be effectively expelled from China,"['Deutsche Welle', 'Www.Dw.Com']","Gauthier, who is based in Beijing and works for the news magazine ""L'Obs,"" is due to leave the Chinese capital on a 1 a.m. flight on Friday (Thursday, 1500 UTC).\n\nSpeaking ahead 